# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path(r"C:\Users\maria\OneDrive\Desktop\Data Analyst Course\Week 7 - Project 1\API Homework\python_api_challenge\Starter_Code\output_data\cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', 
                                      color='Humidity', hover_cols=['City', 'Humidity'], 
                                      tiles='OSM', frame_width=700, frame_height=500)


# Display the map
hvplot.show(map_plot)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df.head()


NameError: name 'ideal_cities_df' is not defined

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
hotel_df = city_data_df[['City', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ""  
# Set parameters to search for a hotel
radius = 5000  # Example radius in meters
params = {
    'apiKey': geoapify_key, 
    'type': 'accommodation',  # Search for accommodations (hotels)
    'limit': 1  # Limit to the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add location, filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'  # Filter for a circle area with the city's location as the center
    params["bias"] = f'proximity:{lng},{lat}'  # Bias results to be closer to the city's location
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Willow Court Motel
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: Carnarvon Motel
sisimiut - nearest hotel: Sømandshjemmet
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: Дом Туриста - Саблино, гостевой дом, миникемпинг
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Gecko Guesthouse
avarua - nearest hotel: Central Motel
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No 

SSLError: HTTPSConnectionPool(host='api.geoapify.com', port=443): Max retries exceeded with url: /v2/places?apiKey=2491d0f675fb4ac0ae6f0226f1b82f95&type=accommodation&limit=1&filter=circle%3A-71.6273%2C-33.0393%2C5000&bias=proximity%3A-71.6273%2C-33.0393 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
# Ensure 'Hotel Name' and 'City' columns are in string format to avoid any issues in plotting
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].astype(str)
hotel_df['City'] = hotel_df['City'].astype(str)  # Replace 'Country' with the actual country column name if different

# Configure the map plot with additional hover information
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity',  # or another size metric
    color='Humidity',  # or another color metric
    hover_cols=['City', 'Humidity', 'Hotel Name'],
    tiles='OSM', frame_width=700, frame_height=500
)

# Display the map
hvplot.show(map_plot)


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : fill_color='Humidity' [no close matches], hatch_color='Humidity' [no close matches], line_color='Humidity' [no close matches] {renderer: GlyphRenderer(id='p1178', ...)}
